bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount fips

In [1]:
from env import get_db_url
import pandas as pd
import numpy as np


In [36]:
url = get_db_url('zillow')
query = '''
SELECT bedroomcnt,
bathroomcnt,calculatedfinishedsquarefeet,
taxvaluedollarcnt,yearbuilt,
taxamount, fips 
FROM properties_2016 
JOIN propertylandusetype
USING(propertylandusetypeid)
WHERE propertylandusedesc = "Single Family Residential";'''
df_2016 = pd.read_sql(query,url)


In [38]:
query = '''
SELECT bedroomcnt,
bathroomcnt,calculatedfinishedsquarefeet,
taxvaluedollarcnt,yearbuilt,
taxamount, fips 
FROM properties_2017
JOIN propertylandusetype
USING(propertylandusetypeid)
WHERE propertylandusedesc = "Single Family Residential";'''
df = pd.concat([df_2016,pd.read_sql(query,url)],axis=0)

In [39]:
#Doing this so I'm not waiting a thousand years for SQL to do its thing.
df.to_csv('data/singlefam.csv',index=False)

In [100]:
df = pd.read_csv('data/singlefam.csv')
datashape = df.shape

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4301225 entries, 0 to 4301224
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 229.7 MB


In [102]:
print(f'df shape: {df.shape[0]} rows x {df.shape[1]} columns')
df.isna().sum()

df shape: 4301225 rows x 7 columns


bedroomcnt                         22
bathroomcnt                        22
calculatedfinishedsquarefeet    18315
taxvaluedollarcnt                 981
yearbuilt                       19990
taxamount                        8838
fips                                0
dtype: int64

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4301225 entries, 0 to 4301224
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 229.7 MB


dropping indices where $null \gt 4$ (i.e. where more than half the features are null)

In [104]:
df = df.dropna(subset=['bedroomcnt','bathroomcnt']).dropna(thresh=4)
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,4.0,3.0,2090.0,615000.0,1973.0,8625.75,6037.0
1,5.0,3.0,2479.0,531642.0,1973.0,6722.67,6037.0
2,4.0,2.0,2057.0,328148.0,1973.0,4160.63,6037.0
3,4.0,3.0,2423.0,297377.0,1978.0,3883.47,6037.0
4,4.0,3.0,2226.0,297001.0,1978.0,3884.27,6037.0


In [105]:
#check if there are half bedrooms (should not be any)
df[(df.bedroomcnt / .5)%2 > 0]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips


There are none, so we will convert the bedroomcnt to int

In [106]:
df.bedroomcnt = df.bedroomcnt.astype(int)
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4300579 entries, 0 to 4301224
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    4300579 non-null  int64  
 1   bathroomcnt                   4300579 non-null  float64
 2   calculatedfinishedsquarefeet  4282902 non-null  float64
 3   taxvaluedollarcnt             4300222 non-null  float64
 4   yearbuilt                     4281221 non-null  float64
 5   taxamount                     4292383 non-null  float64
 6   fips                          4300579 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 262.5 MB


In [107]:
#We'll use 0 as the default value in yearbuilt and convert to int
df.yearbuilt = df.yearbuilt.fillna(0).astype(int)
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4300579 entries, 0 to 4301224
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    4300579 non-null  int64  
 1   bathroomcnt                   4300579 non-null  float64
 2   calculatedfinishedsquarefeet  4282902 non-null  float64
 3   taxvaluedollarcnt             4300222 non-null  float64
 4   yearbuilt                     4300579 non-null  int64  
 5   taxamount                     4292383 non-null  float64
 6   fips                          4300579 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 262.5 MB


Because `taxvaluedollarcnt` has less than 1% na, we will drop this as well

In [108]:
df = df.dropna(subset=['taxvaluedollarcnt','taxamount'])
df.isna().sum()


bedroomcnt                          0
bathroomcnt                         0
calculatedfinishedsquarefeet    14776
taxvaluedollarcnt                   0
yearbuilt                           0
taxamount                           0
fips                                0
dtype: int64

checking what percentage of the data are NaN values in calculatedfinishedsquarefeet

In [119]:
df = df.dropna(subset='calculatedfinishedsquarefeet')
df.shape[0]/datashape[0]

0.9944713424663904

In [1]:
#check that the wrangle function works
import wrangle
df = wrangle.wrangle_zillow()
df.info(show_counts=True)
